Parâmetros iniciais do caderno notebook

In [52]:
base_path = 'C:/projetos/prova-de-conceito/dados/'
#base_path = '/content/drive/My Drive/Colab Notebooks/'


Importando as bibliotecas


In [53]:

import pandas as pd
import time
import pytz
from datetime import datetime
import os
import csv
import requests, zipfile
from xml.etree import ElementTree
import re
import tokenize

class Main:     
    def show_current_time(self):
        ini = time.time()
        tz_SP = pytz.timezone('America/Sao_Paulo') 
        now = datetime.now(tz_SP)
        current_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print("Current Time =", current_time)
        
    def monta_linha(self,artigo,id,txt):
        tipo= artigo.attrib['artType']
        data=artigo.attrib['pubDate']
        emissor=artigo.attrib['artCategory']
        responsavel=""
        cargo=""
        texto = txt.split("\"assina\">")
        if(len(texto)>1):
            texto1 = texto[1].split("</p><p class=\"cargo\">")  
            responsavel = texto1[0].split("<")[0]
            if "cargo" in texto[1] and len(texto1) > 1:
                cargo = texto1[1].split("</p>")[0]
            responsavel=responsavel
            
        return [id,tipo,emissor,data,responsavel,cargo] 

        

print("Versão dopandas: " + pd.__version__)  
  

Versão dopandas: 1.4.2


Gerando arquivos .CSV

In [54]:
#anos = ['2019','2020','2021','2022']
anos = ['2022']
meses = ['01','02','03','04','05','06',\
         '07','08','09','10','11','12']
#meses=[]
secoes= ['01','02','03']
          

fx = Main()

fx.show_current_time()

for ano in anos:
    ini = time.time()
    contador = 0
    count_error = 0  
    pasta = base_path +"dou/" + ano +'/'
    os.chdir(pasta) 
    csvfileunico = open(ano+'.csv','w', newline='', encoding='utf-8')
    csvwriterunico=csv.writer(csvfileunico)
    col_names = ['id','tipo','emissor','data','responsavel','cargo'] 
    csvwriterunico.writerow(col_names)
    print(pasta + ano+'.csv')
    
    for mes in meses:
        inim = time.time()
        for secao in secoes:
            inip = time.time()
            #print('ano=' + ano + ' mes=' + mes + ' secao=' + secao)
            path = base_path +"dou/" + ano+'/'+mes+'/'+secao+'/'
            #print(path)
            os.chdir(path)      
            count_ok = 0
            mypath = path
            file_download = path + 'S' + secao + mes + ano + '.zip'
            isfile = os.path.isfile(file_download) 
            #print(file_download)
            if isfile:  
                with zipfile.ZipFile(file_download, 'r') as myzip:
                    print(file_download)
                    for name in myzip.namelist():
                        with myzip.open(name) as myfile:
                            try:
                                if myfile.name[-3:]  == 'xml':                
                                    id=myfile.name[:-4]  
                                    tree = ElementTree.parse(myfile)  
                                    artigo = tree.getroot().find('article')
                                    texto = artigo.find('body').find('Texto')
                                    linha = fx.monta_linha(artigo,id,texto.text)
                                    contador = contador + 1
                                    csvwriterunico.writerow(linha) 
                                    count_ok = count_ok + 1
                            except:
                                print("Error on parsing ..." + myfile.name)
                                count_error = count_error + 1
                    myzip.close()
    
    csvfileunico.close()

    fim = time.time()

    print("Tempo total: " + ano + ": " + str(fim-ini))
    fx.show_current_time()
    print("qtd OK" , contador)
    print("qtd ERRO" , count_error)


Current Time = 14/09/2022 15:56:39
C:/projetos/prova-de-conceito/dados/dou/2022/2022.csv
C:/projetos/prova-de-conceito/dados/dou/2022/01/01/S01012022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/01/02/S02012022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/01/03/S03012022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/02/01/S01022022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/02/02/S02022022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/02/03/S03022022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/03/01/S01032022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/03/02/S02032022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/03/03/S03032022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/04/01/S01042022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/04/02/S02042022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/04/03/S03042022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/05/01/S01052022.zip
C:/projetos/prova-de-conceito/dados/dou/2022/05/02/S02052022.zip
C